# 🔍 Hunt Splunk - Data Leak Analysis (Offline / Airgapped)

Notebook pour l'analyse de logs Stormshield exportés depuis Splunk.

**Objectif** : identifier des exfiltrations de données, accès suspects, transferts vers services cloud, etc.


In [ ]:
# 📦 Chargement des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipaddress import ip_address, ip_network
from tqdm import tqdm

tqdm.pandas()

In [ ]:
# 📁 Chargement du dataset Splunk exporté en CSV
df = pd.read_csv('../datasets/stormshield_logs.csv')  # adapte le chemin si besoin
df['_time'] = pd.to_datetime(df['_time'])
df.head()

In [ ]:
# 🧹 Nettoyage et enrichissement : IP interne/externe
PRIVATE_CIDRS = [
    ip_network('10.0.0.0/8'),
    ip_network('172.16.0.0/12'),
    ip_network('192.168.0.0/16')
]

def classify_ip(ip):
    try:
        ip_obj = ip_address(ip)
        if any(ip_obj in cidr for cidr in PRIVATE_CIDRS):
            return 'Interne'
        return 'Externe'
    except:
        return 'Invalide'

df['ip_type'] = df['src_ip'].progress_apply(classify_ip)

In [ ]:
# 📊 Statistiques globales
df['total_bytes'] = df['bytes_out'] + df['bytes_in']
df['total_Go'] = (df['total_bytes'] / 1024**3).round(3)
df.groupby(['src_ip', 'ip_type'])['total_Go'].sum().sort_values(ascending=False).head(20)

In [ ]:
# 📈 Volume horaire par IP source
df['hour'] = df['_time'].dt.floor('H')
pivot = df.pivot_table(index='hour', columns='src_ip', values='total_bytes', aggfunc='sum').fillna(0)
pivot = pivot / 1024**2  # en Mo
pivot.plot(figsize=(15,6), title='Volume horaire par IP source (Mo)')
plt.ylabel('Mo')
plt.grid(True)
plt.tight_layout()

In [ ]:
# 🧪 Filtrage des services de transfert (Dropbox, Mega, Wetransfer, etc.)
cloud_services = ['dropbox.com', 'mega.nz', 'wetransfer.com', 'transfer.sh', 'anonfiles.com', 'file.io', 'gofile.io']
df_cloud = df[df['url_domain'].isin(cloud_services)]
df_cloud[['src_ip', 'url_domain', 'total_Go']].sort_values(by='total_Go', ascending=False).head(20)

Tu peux enrichir ce notebook avec :
- un mapping IP → utilisateur (depuis Sysmon ou autre)
- une lookup manuelle pour marquer les postes sensibles
- une analyse par `dest_ip` ou `port_type` si tu les as
- des clusters de comportement avec `scikit-learn`